In [ ]:
import sys
sys.path.append('../scripts')

# import utils,analysis,test_features
# import SCCML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots

In [ ]:
ICC_df = pd.read_pickle('/home/jinzhuo/ww_dataset/SCCML/data/ICC_rms.pkl')

In [ ]:
def plot_avg_spec(data):
    types = list(set(data['type']))
    fig,axes=subplots(1,len(types),figsize=(8*len(types),4))
    ax = axes.ravel()
    
    mzs = np.array(data.columns[data.columns!='type'])
    print(mzs.shape)
    for i in range(len(types)):
        
        ax[i].stem(mzs,data[data.columns[data.columns!='type']][data['type']==types[i]].mean(0),label=types[i],basefmt='darkslateblue',markerfmt=' ',linefmt='darkslateblue',
                  use_line_collection=True)
        ax[i].set_ylabel('Average Intensity',fontsize=12)
        ax[i].set_xlabel('m/z',fontsize=12)
        ax[i].legend()
    fig.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'average_mz.svg',format = 'svg',dpi = 300)
    
# plot_avg_spec(data=HIP_CER_df)
plot_avg_spec(data=ICC_df)
# plot_avg_spec(data=SIMS_df)

In [ ]:
from umap import UMAP
import matplotlib.pyplot as plt
umap = UMAP()
data = ICC_df.drop(['type'],axis=1).values
embedding = umap.fit_transform(data)
x_coords = [x for x, y in embedding]
y_coords = [y for x, y in embedding]
colors = ['red', 'blue']
type_list = list(ICC_df['type'])
type_set = list(set(type_list))
label_added = {}
for i in range(len(type_list)):
    if type_list[i] == type_set[0]:
        num = 0
    else:
        num = 1
    if type_set[num] not in label_added:
    # plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        label_added[type_set[num]] = True
    else:
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],alpha=1,s = 2)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(loc = 'upper right')
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'umap_icc.svg',format = 'svg',dpi = 300)


In [ ]:
from scipy.stats import ranksums
def rank_sum_test(x, y, features, N):
    dim = x.shape[1]
    sample_num = min(x.shape[0], y.shape[0])
    S = []
    P = []
    for i in range(dim):
        s, p = ranksums(x[:, i], y[:, i])
        S.append(s)
        P.append(p)

    S = np.asarray(S)
    P = np.asarray(P)
    # print(P.argsort().shape)

    P_order = P.argsort()[-N:]
    # P_ranks = P_order.argsort()
    feature_order = features[P_order]

    return P_order, feature_order, P
def rank_sum(data,groups):
    
    x = data.loc[data['type']==groups[0], data.columns != 'type'].values
    y = data.loc[data['type']==groups[1], data.columns != 'type'].values
    
    mzs = np.array(data.columns[data.columns!='type'])
        
    O,F,P=rank_sum_test(x,y,mzs,10000)
    # plt.figure(figsize=(5,3))
    # plt.hist(np.log10(P),30)
    # plt.xlabel('log 10 p-value')
    # plt.ylabel('number of mz features')
    return O,F,P
O,F,P = rank_sum(ICC_df,['Astrocytes','Neurons'])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
plt.hist(P,100)
plt.ylabel('number of features')
plt.xlabel('P-Value')
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'rank_sum_icc.svg',format = 'svg',dpi = 300)

# plt.set_ylabel('number of features',fontsize=14)
# ax = axes.ravel()

In [ ]:
count = 0
for i in range(P.shape[0]):
    if P[i]<=0.05:
        count = count+1
print(count,P.shape[0],count/P.shape[0])
Select_feature_ranksum = F[:50]
Select_feature_ranksum_df = ICC_df[Select_feature_ranksum]

In [ ]:
from umap import UMAP
import matplotlib.pyplot as plt
umap = UMAP()
data = Select_feature_ranksum_df.values
embedding = umap.fit_transform(data)
x_coords = [x for x, y in embedding]
y_coords = [y for x, y in embedding]
colors = ['red', 'blue']
type_list = list(ICC_df['type'])
type_set = list(set(type_list))
label_added = {}
for i in range(len(type_list)):
    if type_list[i] == type_set[0]:
        num = 0
    else:
        num = 1
    if type_set[num] not in label_added:
    # plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        label_added[type_set[num]] = True
    else:
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],alpha=1,s = 2)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(loc = 'upper right')
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'umap_icc_ranksum.svg',format = 'svg',dpi = 300)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
ICC_df['type']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def split_data(data,test_size,random_state):

    X_train, X_test, y_train, y_test = train_test_split(data.drop('type',axis = 1).values, data['type'].values, 
                                                        test_size=test_size,
                                                        random_state=random_state)
    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    data_dict = {'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test}

    return data_dict

data_dict_icc = split_data(data=ICC_df,test_size=0.2,random_state=11)

In [ ]:
data_dict_icc['y_train']
le.fit_transform(data_dict_icc['y_train'])

In [ ]:
import xgboost
from sklearn.metrics import classification_report 
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, n_estimators=600, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
def train_model(model,data_dict):
    
    model.fit(data_dict['X_train'],data_dict['y_train'])

    y_pred = model.predict(data_dict['X_test'])
    prob = model.predict_proba(data_dict['X_test'])
    report_dict = classification_report(data_dict['y_test'], y_pred, output_dict=True)
    
    return model, y_pred, prob,report_dict

model, y_pred, prob,report_dict = train_model(model,data_dict_icc)



In [ ]:
fig,axes = subplots(1,2,figsize=(18,4))
ax = axes.ravel()
ax[0].hist(prob[np.where(data_dict_icc['y_test']==0)[0],0],bins=30)
ax[0].set_title('Neurons',fontsize=15)
ax[1].hist(prob[np.where(data_dict_icc['y_test']==1)[0],1],bins=30)
ax[1].set_title('Astrocytes',fontsize=15)
ax[0].set_ylabel('number of cells',fontsize=15)
ax[1].set_ylabel('number of cells',fontsize=15)
ax[0].set_xlabel('predicted probability',fontsize=15)
ax[1].set_xlabel('predicted probability',fontsize=15)
ax[0].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[1].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[0].legend(fontsize=11,loc=2)
ax[1].legend(fontsize=11)
print(prob.shape[0])
y_test_label = list(data_dict_icc['y_test'])
len(list(data_dict_icc['y_test']))
count = 0
for i in range(len(list(y_test_label))):
    
    y_pre = 1
    if prob[i,0] > 0.5:
        y_pre = 0
    else:
        y_pre = 1
    if y_pre == y_test_label[i]:
        count = count+1
    # print(prob[i,0],y_test_label[i])
print(count)
print("accuracy",count/prob.shape[0])
fig.suptitle("XGBOOST")
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'xgboost.svg',format = 'svg',dpi = 300)



In [ ]:
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from shap import TreeExplainer, DeepExplainer, KernelExplainer, LinearExplainer
model_svc = SVC(kernel='linear',C=1, max_iter=10000,random_state=19,probability = True)
model_lda = LinearDiscriminantAnalysis()
model_knn = KNeighborsClassifier(n_neighbors=5, algorithm='auto',leaf_size=30, metric='minkowski', n_jobs=8)


In [ ]:
model_knn.fit(data_dict_icc['X_train'],data_dict_icc['y_train'])
y_pred = model_knn.predict(data_dict_icc['X_test'])
prob = model_knn.predict_proba(data_dict_icc['X_test'])
report_dict = classification_report(data_dict_icc['y_test'], y_pred, output_dict=True)
fig,axes = subplots(1,2,figsize=(18,4))
ax = axes.ravel()
ax[0].hist(prob[np.where(data_dict_icc['y_test']==0)[0],0],bins=30)
ax[0].set_title('Neurons',fontsize=15)
ax[1].hist(prob[np.where(data_dict_icc['y_test']==1)[0],1],bins=30)
ax[1].set_title('Astrocytes',fontsize=15)
ax[0].set_ylabel('number of cells',fontsize=15)
ax[1].set_ylabel('number of cells',fontsize=15)
ax[0].set_xlabel('predicted probability',fontsize=15)
ax[1].set_xlabel('predicted probability',fontsize=15)
ax[0].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[1].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[0].legend(fontsize=11,loc=2)
ax[1].legend(fontsize=11)
print(prob.shape[0])
y_test_label = list(data_dict_icc['y_test'])
len(list(data_dict_icc['y_test']))
count = 0
for i in range(len(list(y_test_label))):
    
    y_pre = 1
    if prob[i,0] > 0.5:
        y_pre = 0
    else:
        y_pre = 1
    if y_pre == y_test_label[i]:
        count = count+1
    # print(prob[i,0],y_test_label[i])
print(count)
print("accuracy",count/prob.shape[0])
fig.suptitle("KNN")
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'knn.svg',format = 'svg',dpi = 300)


In [ ]:
model_lda.fit(data_dict_icc['X_train'],data_dict_icc['y_train'])
y_pred = model_lda.predict(data_dict_icc['X_test'])
prob = model_lda.predict_proba(data_dict_icc['X_test'])
report_dict = classification_report(data_dict_icc['y_test'], y_pred, output_dict=True)
fig,axes = subplots(1,2,figsize=(18,4))
ax = axes.ravel()
ax[0].hist(prob[np.where(data_dict_icc['y_test']==0)[0],0],bins=30)
ax[0].set_title('Neurons',fontsize=15)
ax[1].hist(prob[np.where(data_dict_icc['y_test']==1)[0],1],bins=30)
ax[1].set_title('Astrocytes',fontsize=15)
ax[0].set_ylabel('number of cells',fontsize=15)
ax[1].set_ylabel('number of cells',fontsize=15)
ax[0].set_xlabel('predicted probability',fontsize=15)
ax[1].set_xlabel('predicted probability',fontsize=15)
ax[0].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[1].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[0].legend(fontsize=11,loc=2)
ax[1].legend(fontsize=11)
print(prob.shape[0])
y_test_label = list(data_dict_icc['y_test'])
len(list(data_dict_icc['y_test']))
count = 0
for i in range(len(list(y_test_label))):
    
    y_pre = 1
    if prob[i,0] > 0.5:
        y_pre = 0
    else:
        y_pre = 1
    if y_pre == y_test_label[i]:
        count = count+1
    # print(prob[i,0],y_test_label[i])
print(count)
print("accuracy",count/prob.shape[0])
fig.suptitle("LDA")
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'lda.svg',format = 'svg',dpi = 300)




In [ ]:
model_svc.fit(data_dict_icc['X_train'],data_dict_icc['y_train'])
y_pred = model_svc.predict(data_dict_icc['X_test'])
prob = model_svc.predict_proba(data_dict_icc['X_test'])
report_dict = classification_report(data_dict_icc['y_test'], y_pred, output_dict=True)
fig,axes = subplots(1,2,figsize=(18,4))
ax = axes.ravel()
ax[0].hist(prob[np.where(data_dict_icc['y_test']==0)[0],0],bins=30)
ax[0].set_title('Neurons',fontsize=15)
ax[1].hist(prob[np.where(data_dict_icc['y_test']==1)[0],1],bins=30)
ax[1].set_title('Astrocytes',fontsize=15)
ax[0].set_ylabel('number of cells',fontsize=15)
ax[1].set_ylabel('number of cells',fontsize=15)
ax[0].set_xlabel('predicted probability',fontsize=15)
ax[1].set_xlabel('predicted probability',fontsize=15)
ax[0].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[1].axvline(0.5,linestyle='-.',c='r',label='binary classification threshold')
ax[0].legend(fontsize=11,loc=2)
ax[1].legend(fontsize=11)
# print(prob.shape[0])
y_test_label = list(data_dict_icc['y_test'])
len(list(data_dict_icc['y_test']))
count = 0
for i in range(len(list(y_test_label))):
    
    y_pre = 1
    if prob[i,0] > 0.5:
        y_pre = 0
    else:
        y_pre = 1
    if y_pre == y_test_label[i]:
        count = count+1
    # print(prob[i,0],y_test_label[i])
print(count)
print("accuracy",count/prob.shape[0])
fig.suptitle("SVM")
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'svm.svg',format = 'svg',dpi = 300)




In [ ]:
import shap
shap_explainer = TreeExplainer(model)
shap_vals = shap_explainer.shap_values(ICC_df.drop(['type'],axis =1))
shap_vals.shape
shap.summary_plot(shap_vals, ICC_df.drop(['type'],axis =1))

In [ ]:
from adjustText import adjust_text
import shap
shap_explainer = TreeExplainer(model)
shap_vals = shap_explainer.shap_values(ICC_df.drop(['type'],axis =1))
fig1,axes = subplots(figsize=(12,4))
features = ICC_df.columns[:-1]
feature_num_shown = 20
    
g=axes.stem([float(x) for x in features],shap_vals.mean(axis=0),markerfmt=' ',linefmt='k')
axes.get_yaxis().set_ticks([])
shap_vals_index_ranked = np.argsort(shap_vals.mean(axis=0))[::-1]
shap_vals_ranked = shap_vals.mean(axis=0)[shap_vals_index_ranked]
#axes.spines['right'].set_visible(False)
#axes.spines['top'].set_visible(False)

texts = []
axes.scatter(features[shap_vals_index_ranked[:feature_num_shown]],shap_vals_ranked[:feature_num_shown],color='b',s=35,marker='v')
axes.scatter(features[shap_vals_index_ranked[-feature_num_shown:]],shap_vals_ranked[-feature_num_shown:],color='orange',s=35,marker='s')
for i in range(feature_num_shown):
    texts.append(plt.text(float(features[shap_vals_index_ranked[i]]),shap_vals_ranked[i],float(features[shap_vals_index_ranked[i]]),fontsize=12))
    texts.append(plt.text(float(features[shap_vals_index_ranked[-i-1]]),shap_vals_ranked[-i-1],float(features[shap_vals_index_ranked[-i-1]]),fontsize=12))

#         axes.annotate(format(float(features[shap_vals_index_ranked[i]]),'.2f'), xy=(float(features[shap_vals_index_ranked[i]]),shap_vals_ranked[i]),fontsize=10)
#         axes.annotate(format(float(features[shap_vals_index_ranked[-i-1]]),'.2f'), xy=(float(features[shap_vals_index_ranked[-i-1]]),shap_vals_ranked[-i]),fontsize=10)
axes.set_xlabel('m/z',fontsize=12)
axes.set_ylabel('mean SHAP values',fontsize=12)
axes.legend()
adjust_text(texts)
# if if_summary:
fig2,axes = subplots(figsize=(12,4))
shap.summary_plot(shap_vals,ICC_df.drop(['type'],axis =1),max_display=15)
fig2.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'featureimportance.svg',format = 'svg',dpi = 300)
fig1.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'meanshapvalue.svg',format = 'svg',dpi = 300)

# shap_values = explainer.shap_values(X_test.iloc[0,:])
# shap.force_plot(explainer.expected_value[0], shap_values[0], X_test.iloc[0,:])

In [ ]:
# features[shap_vals_index_ranked]
contrib_xgb_best = shap_vals
shap_ranked_index = np.argsort(abs(contrib_xgb_best).mean(0))[::-1]

X_df = pd.DataFrame(ICC_df.drop('type',axis=1))
X_df.columns = ICC_df.columns[:-1]

contrib_df = pd.DataFrame()
contrib_df['mass'] = features[shap_ranked_index]
contrib_df['mean SHAP'] = np.around(abs(contrib_xgb_best).mean(0),4)[shap_ranked_index]
contrib_df['mean SHAP'] = np.round(contrib_df['mean SHAP']/contrib_df['mean SHAP'].max(),4)
a=X_df.values*contrib_xgb_best
contrib_df.loc[a.mean(0)[shap_ranked_index]>0,'contribute to which type']='Neurons'
contrib_df.loc[a.mean(0)[shap_ranked_index]<0,'contribute to which type']='Astrocytes'
contrib_df = contrib_df[contrib_df['mean SHAP']!=0]
contrib_df.head(10)

In [ ]:
from umap import UMAP
import matplotlib.pyplot as plt
umap = UMAP()
data = ICC_df.drop(['type'],axis=1).values
embedding = umap.fit_transform(shap_vals)
x_coords = [x for x, y in embedding]
y_coords = [y for x, y in embedding]
colors = ['red', 'blue']
type_list = list(ICC_df['type'])
type_set = list(set(type_list))
label_added = {}
for i in range(len(type_list)):
    if type_list[i] == type_set[0]:
        num = 0
    else:
        num = 1
    if type_set[num] not in label_added:
    # plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        label_added[type_set[num]] = True
    else:
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],alpha=1,s = 2)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(loc = 'upper right')
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'umap_shapvalue.svg',format = 'svg',dpi = 300)


In [ ]:
sel = shap_vals[:,shap_ranked_index[:3]]
from umap import UMAP
import matplotlib.pyplot as plt
umap = UMAP()
data = ICC_df.drop(['type'],axis=1).values
embedding = umap.fit_transform(sel)
x_coords = [x for x, y in embedding]
y_coords = [y for x, y in embedding]
colors = ['red', 'blue']
type_list = list(ICC_df['type'])
type_set = list(set(type_list))
label_added = {}
for i in range(len(type_list)):
    if type_list[i] == type_set[0]:
        num = 0
    else:
        num = 1
    if type_set[num] not in label_added:
    # plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],label = type_set[num],alpha=1,s = 2)
        label_added[type_set[num]] = True
    else:
        plt.scatter(x_coords[i], y_coords[i], c=colors[num],alpha=1,s = 2)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(loc = 'upper right')
plt.savefig('/home/jinzhuo/ww_dataset/DECOMPOSITION/Figure1/'+'umap_shapvalue_select50.svg',format = 'svg',dpi = 300)
